In [1]:
import os
import tensorflow as tf
from tensorflow import keras

In [2]:
(train_images, train_labels),(test_images,test_labels) = tf.keras.datasets.mnist.load_data()
train_labels = train_labels[:1000]
test_labels = test_labels[:1000]

train_images = train_images[:1000].reshape(-1,28*28)/255.0
test_images = test_images[:1000].reshape(-1,28*28)/255.0

11490434/11490434 [==============================] - 1s 0us/step


In [9]:
def create_model():
    model = tf.keras.models.Sequential([
        keras.layers.Dense(512,activation='relu',input_shape=(784,)),
        keras.layers.Dropout(0.2),
        keras.layers.Dense(10)
    ])
    model.compile(optimizer='adam',
                  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
                  metrics=[tf.metrics.SparseCategoricalAccuracy()])
    return model

model = create_model()
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 512)               401920    
                                                                 
 dropout_3 (Dropout)         (None, 512)               0         
                                                                 
 dense_7 (Dense)             (None, 10)                5130      
                                                                 
Total params: 407,050
Trainable params: 407,050
Non-trainable params: 0
_________________________________________________________________


In [10]:
#użycie wołania zwrotnego w punkcie kontrolnym

checkpoint_path = "training_1/cp.ckpt"
checkpoint_dir = os.path.dirname(checkpoint_path)

#utworzenie wywołania zwrotnego (callback) z zapisem wag
cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_path,
                                                 save_weights_only=True,
                                                 verbose=1)

model.fit(train_images,
          train_labels,
          epochs=10,
          validation_data=(test_images,test_labels),
          callbacks=[cp_callback])

Epoch 1/10
32/32 [==============================] - ETA: 0s - loss: 1.1315 - sparse_categorical_accuracy: 0.7020
Epoch 1: saving model to training_1/cp.ckpt
32/32 [==============================] - 4s 119ms/step - loss: 1.1315 - sparse_categorical_accuracy: 0.7020 - val_loss: 0.7359 - val_sparse_categorical_accuracy: 0.7770
Epoch 2/10
32/32 [==============================] - ETA: 0s - loss: 0.4289 - sparse_categorical_accuracy: 0.8810
Epoch 2: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 23ms/step - loss: 0.4289 - sparse_categorical_accuracy: 0.8810 - val_loss: 0.5074 - val_sparse_categorical_accuracy: 0.8460
Epoch 3/10
32/32 [==============================] - ETA: 0s - loss: 0.2744 - sparse_categorical_accuracy: 0.9360
Epoch 3: saving model to training_1/cp.ckpt
32/32 [==============================] - 1s 25ms/step - loss: 0.2744 - sparse_categorical_accuracy: 0.9360 - val_loss: 0.4594 - val_sparse_categorical_accuracy: 0.8600
Epoch 4/10
32/32 [======

In [11]:
os.listdir(checkpoint_dir)

['checkpoint', 'cp.ckpt.data-00000-of-00001', 'cp.ckpt.index']

In [12]:
model = create_model()
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'niewytrenowany model, accuracy: {100*acc:5.2f} %')

32/32 - 0s - loss: 2.3285 - sparse_categorical_accuracy: 0.1190 - 186ms/epoch - 6ms/step
niewytrenowany model, accuracy: 11.90 %


In [13]:
model.load_weights(checkpoint_path)
loss,acc = model.evaluate(test_images,test_labels,verbose=2)
print(f'model z odtworzonymi wagami z obcego modelu, accuracy: {100*acc:5.2f} %')

32/32 - 0s - loss: 0.4194 - sparse_categorical_accuracy: 0.8580 - 77ms/epoch - 2ms/step
model z odtworzonymi wagami z obcego modelu, accuracy: 85.80 %
